In [ ]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# K-Means Clustering



![](withinclustervariation.png)

```{prf:algorithm} K-Means Clustering
:label: k-means

**Inputs** Given a number of clusters $K$

**Output** A partitioning of the dataset into $K$ distinct clusters.

1. Initialize $K$ random centroids by randomly selecting $K$ points from the dataset.
2. While the clusters continue to change (the algorithm hasn't converged):

	1. Construct clusters by associating each point with closest centroid
	2. Calculate new centroids for each cluster by averaging the observations for each feature.

**Note:** The final clustering could depend on where the centroids are initialized.
```


{prf:ref}`k-means`

The animation below shows how the algorithm initializes and updates until convergence.

In [4]:
from IPython.display import Image, display
X = Image(url='kmeans_animation.gif')
display(X)

Notice, that if we change the initial centroids, it can result in different clusters.

![](kmeans_animation_2.gif)